# Model

In [1]:
import pandas as pd
from database import engine

from IPython.display import HTML

## Loading data

- We only load records that belong to the `view` category, because every like starts with view, and we have a `target` column that marks those views that lead to likes.

In [2]:
records_count = 1000

user_data = pd.read_sql(
    f"SELECT * FROM public.user_data LIMIT {records_count};",
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    f"SELECT * FROM public.post_text_df LIMIT {records_count};",
    con = engine,
    index_col = "post_id"
)
feed_data = pd.read_sql(
    f"SELECT * FROM public.feed_data limit {records_count};",
    con = engine
)

# joined data is where each user 
# is matched with all his actions
query = f"""
SELECT 
    public.feed_data.timestamp,
    public.feed_data.target,
    public.user_data.gender,
    public.user_data.age,
    public.user_data.country,
    public.user_data.city,
    public.user_data.exp_group,
    public.user_data.os,
    public.user_data.source,
    public.post_text_df.text,
    public.post_text_df.topic
FROM public.feed_data
LEFT JOIN public.user_data
    ON public.feed_data.user_id = public.user_data.user_id
LEFT JOIN public.post_text_df
    ON public.feed_data.post_id = public.post_text_df.post_id
WHERE
    public.feed_data.action='view'
LIMIT {records_count};
"""
joined_data = pd.read_sql(query, con = engine)

In [3]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data,
    "Joined data" : joined_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business


,timestamp,user_id,post_id,action,target
0,2021-10-30 20:57:49,1628,3144,view,0
1,2021-10-30 20:58:45,1628,5228,view,1
2,2021-10-30 21:00:50,1628,5228,like,0
3,2021-10-30 21:00:52,1628,1560,view,0
4,2021-10-30 21:01:03,1628,74,view,0


,timestamp,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-10-30 20:57:49,0,0,20,Russia,Moscow,3,Android,ads,States with most number of deaths and more tha...,covid
1,2021-10-30 20:58:45,1,0,20,Russia,Moscow,3,Android,ads,Alien was excellent. Many writers tried to cop...,movie
2,2021-10-30 21:00:52,0,0,20,Russia,Moscow,3,Android,ads,Beattie return calms attack fears\n\nEverton s...,sport
3,2021-10-30 21:01:03,0,0,20,Russia,Moscow,3,Android,ads,France Telecom gets Orange boost\n\nStrong gro...,business
4,2021-10-30 21:01:23,0,0,20,Russia,Moscow,3,Android,ads,"@KamalaHarris @JoeBiden acceptance speech, wal...",covid


## Transformation

In [4]:
X = joined_data.copy()

y = X["target"]
X.drop("target", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

X['gender'] = X['gender'].astype("O")

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

In [59]:
numeric_columns = list(X.select_dtypes("number").columns)
categorical_columns = [
    'gender', 'country', 'city', 'os', 'source', 'topic'
]
text_columns = 'text'

In [38]:
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)

In [74]:
my_transfromer = ColumnTransformer([
    ("stand_scaler", StandardScaler(), numeric_columns),
    (
        "one_hot_encod", 
        OneHotEncoder(
            categories = list(X[categorical_columns].apply(lambda col: list(col.unique())))
        ), 
        categorical_columns
    ),
    ("tf_idf_vector", TfidfVectorizer(max_features = 10), text_columns)
])

In [77]:
from sklearn.ensemble import GradientBoostingClassifier

In [79]:
GradientBoostingClassifier().fit(
    my_transfromer.fit_transform(X), y
)

GradientBoostingClassifier()